In [2]:
import os
os.chdir('../../data')

In [3]:
ls

categorical_encoding.csv  final_data.csv            sf-crime/
encoded_data.csv          kangwon.csv               submission1.csv


In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('encoded_data.csv', encoding='utf-8')
data.shape

(24697792, 12)

In [5]:
data['날짜'] = data['날짜'].astype(str)

In [6]:
data['연도'] = data['날짜'].str[:4]
data['월'] = data['날짜'].str[4:]
del data['날짜']

In [7]:
data['연도'] = data['연도'].astype(int)
data['월'] = data['월'].astype(int)

In [8]:
data.head(1)

,가맹점주소,가맹점업종_이름,가맹점업종,사용자거주시도,연령대,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액,남자,여자,연도,월
0,강원,건강보조식품 소매업,0.000115,0.7577,0.09523,0.0961,4,4,311200,1,0,2019,1


In [9]:
temp_cols = data.columns.tolist()
temp_cols.remove('가맹점주소')
temp_cols.remove('카드사용금액')
print(temp_cols)

['가맹점업종_이름', '가맹점업종', '사용자거주시도', '연령대', '가구생애주기', '카드사용인원수(환불사용자포함)', '카드사용건수(환불사용자제외)', '남자', '여자', '연도', '월']


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPRegressor
from itertools import product

def encoder_func(df):
    encoders = {}
    cols = ['가맹점주소', '가맹점업종_이름']
    for col in cols:
        encoder = LabelEncoder()
        encoders[col] = encoder
    for column in encoders.keys():
        encoder = encoders[column]
        df[column] = encoder.fit_transform(df[column])
    # 인코딩된 df이랑 칼럼별 인코더 담고있는 dict객체반환(추후에 inverse용)
    return df, encoders
        

# 지역별 모델링 반복 돌리기
region_model_dict = {}
region_forecasting = pd.DataFrame(columns=['날짜','가맹점주소',
                                          '가맹점업종_이름','카드사용금액'])

city_lists = data['가맹점주소'].unique()

for city in city_lists:
    # 지역별로 모델링 되는지 체크용 출력
    print(city)
    # 지역별로 샘플링해서 하나의 데이터프레임으로 할당
    sampling = data.query('가맹점주소 == @city')
    
    # 지역별로 주어진 기간내에 존재하지 않았던 업종별 따로 리스트로 담기
    none_upjong = list( set(data['가맹점업종_이름'].unique()) - set(sampling['가맹점업종_이름'].unique()) )
    
    # 샘플링된 지역별 데이터프레임에 '가맹점주소'랑 '가맹점업종_이름' 인코딩 시키기
    sampling, encoders = encoder_func(sampling)
    
    ### train data는 일부 데이터만 활용--->생각해보자..
    train_num = sampling.sample(frac=1, random_state=42)
    # 지역별로 모델링하니까 '가맹점주소'도 지워주기, '날짜'도 지워줘야 할까..!?
    train_x = train_num.drop(['연도','월','가맹점주소','카드사용금액'],
                            axis=1)
    train_y = np.log1p(train_num['카드사용금액'])
    
    model = MLPRegressor()
    model.fit(train_x, train_y)
    # 학습된 모델 region_model_dict에다가 담기
    region_model_dict[city+' model'] = model
    
    # 예측 탬플릿 생성
    year_temp = [2020]
    month_temp = [4,7]
    upjong_temp = sampling['가맹점업종_이름'].unique()
    upjong_w_temp = sampling['가맹점업종'].unique()
    home_temp = sampling['사용자거주시도'].unique()
    age_temp = sampling['연령대'].unique()
    cycle_temp = sampling['가구생애주기'].unique()
    usage_temp = sampling['카드사용인원수(환불사용자포함)'].unique()
    usage2_temp = sampling['카드사용건수(환불사용자제외)'].unique()
    male_temp = sampling['남자'].unique()
    female_temp = sampling['여자'].unique()
    
    comb_list = [upjong_temp, upjong_w_temp, home_temp, age_temp,
                cycle_temp, usage_temp, usage2_temp, male_temp,
                female_temp, year_temp, month_temp]
    
    forecasting_temp = np.array(list(product(*comb_list)))
    forecasting_temp = pd.DataFrame(data=forecasting_temp,
                                   columns=temp_cols)
    
    pred = model.predict(forecasting_temp)
    pred = np.expm1(pred)
    forecasting_temp['예측금액'] = np.round(pred, 0)
    forecasting_temp['날짜'] = forecasting_temp['연도']*100 + forecasting_temp['월']
    forecasting_temp = forecasting_temp[['날짜', '가맹점업종_이름',
                                        '카드사용금액']]
    # 첫번째 column 자리에다가 '가맹점주소' 이름으로 city 값들 추가
    forecasting_temp.insert(1, '가맹점주소', city)
    # 날짜, 지역, 업종별로 사용금액 합계 집계하기
    forecasting_temp = forecasting_temp.groupby(['날짜','가맹점주소',
                                                '가맹점업종_이름']).sum().reset_index(drop=False)
    forecasting_temp['가맹점업종_이름'] = encoders['가맹점업종_이름'].inverse_transform(forecasting_temp['가맹점업종_이름'])
    
    # 지역별로 존재하지 않았던 업종들 0으로 넣기
    for none in none_upjong:
        for i in range(2):
            date = [202004, 202007]
            tmp = pd.DataFrame([[date[i], city, none, 0]],
                              columns=forecasting_temp.columns)
            forecasting_temp = pd.concat([forecasting_temp, tmp],
                                        axis=0)
    
    region_forecasting = pd.concat([region_forecasting,
                                forecasting_temp], axis=0)
# 지역별 모델담은 dict 출력
print(region_model_dict)    

강원


/Users/younghun/opt/anaconda3/envs/venvforpython/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
